In [1]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta

import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
2024-10-15 20:31:33,903 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
start_date = datetime.now() - timedelta(hours=48)
print(start_date.strftime("%Y-%m-%d"))

2024-10-13


In [4]:
end_date = datetime.now() - timedelta(hours=24)
print(end_date.strftime("%Y-%m-%d"))

2024-10-14


In [5]:
feature_view = fs.get_feature_view('amd_stock_fv', 2)
feature_view.init_batch_scoring(training_dataset_version=2)

In [6]:
df = feature_view.get_batch_data(start_time=start_date, end_time=end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.83s) 


In [7]:
df.head()

,date,f_1__open
0,2024-10-14 00:00:00+00:00,167.77


In [11]:
import joblib

the_model = mr.get_model("ProphetModel", version=5)
model_dir = the_model.download()

model = joblib.load(model_dir + "/Prophet_model.pkl")

In [16]:
# Remove timezone information from 'ds' column
df['ds'] = pd.to_datetime(df['ds']).dt.tz_localize(None)


In [17]:
print(df['ds'].dtype)


datetime64[us]


In [18]:
# Predict using the model
predictions = model.predict(df)

# Display predictions
print(predictions[['ds', 'yhat']].head())


          ds       yhat
0 2024-10-14  59.316814


In [19]:
predictions = model.predict(df)

In [20]:
predictions 

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-10-14,-69.240215,48.834273,69.426685,-69.240215,-69.240215,128.557029,128.557029,128.557029,38.591054,...,136.014934,136.014934,136.014934,-46.048959,-46.048959,-46.048959,0.0,0.0,0.0,59.316814


In [22]:
# Assign the 'yhat' column from 'predictions' to 'df'
df['predictions'] = predictions['yhat'].values


In [23]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.


In [34]:
results_fg = fs.get_or_create_feature_group(
    name= 'stock_prediction_results',
    version = 2,
    description = 'Predction of AMD close stock price',
    primary_key = ['f_1__open'],
    event_time = ['ds'],
    online_enabled = False,
)

In [35]:
results_fg.insert(df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fg/1283405


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: stock_prediction_results_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/stock_prediction_results_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1c95d027dd0>, None)